In [1]:
import json
import pandas as pd

import numpy as np
import glob

In [2]:
members = pd.read_hdf("list_of_members.h5", "members")
members["full_name_"] = members["first_name"] + " " + members["last_name"]
members["last_name_lower"] = members["last_name"].str.lower()

In [3]:
state_names = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AS": "American Samoa",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "DC": "District Of Columbia",
    "FM": "Federated States Of Micronesia",
    "FL": "Florida",
    "GA": "Georgia",
    "GU": "Guam",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MH": "Marshall Islands",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "MP": "Northern Mariana Islands",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PW": "Palau",
    "PA": "Pennsylvania",
    "PR": "Puerto Rico",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VI": "Virgin Islands",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming"
}

state_names = {v: k for k, v in state_names.items()}

In [4]:
def speech_to_df(speech_filename, chamber='house'):
    """
    This function reads in a json and converts it to a pandas dataframe.
    It also tries to find missing bioguide ids if they exist in the data.
    """
    from pandas.io.json import json_normalize
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process

    # Load json from file
    speech_json = json.load(open(speech_filename))
    # flatten json into dataframes
    speeches = json_normalize(speech_json["content"])
    metadata = json_normalize(speech_json).drop(["content"], axis=1)
    
    # Add filename to dataframe
    #speeches["file"] = speech_filename
    
   # Join dataframes together, duplicating info over all rows
    df = speeches.join(pd.DataFrame(np.repeat(metadata.values, len(speeches), axis=0), columns=metadata.columns))
    # Create a date column
    df["date"] = pd.to_datetime(df["header.day"] + " " + df["header.month"] + " " + df["header.year"])
    # Make lower case
    df["header.chamber"] = df["header.chamber"].str.lower()
    
    # Keep only one chamber and only only speech rows
    speech_df = df.loc[(df["kind"] == 'speech') & (df["header.chamber"] == chamber.lower())].copy()
    
    ## If there are any speeches without a bioguide id, let's try to find the appropriate person
    if ("speaker_bioguide" in speech_df.columns) and (len(speech_df.query("speaker_bioguide == 'None'")) > 0):
        # Assume that there is only one unknown speaker per dataframe
        unknown_speaker = speech_df.loc[speech_df["speaker_bioguide"] == "None"].iloc[0]
        # Get the title row which may contain a full name
        title_rows = df.query("kind=='title'")

        if len(title_rows) > 0:
            #print(speech_filename, " Num. rows: ", len(title_rows))
            title_row = title_rows.iloc[0]["text"].split("\n")[0].strip()

            # Get list of possible members based on last name
            if " of " in unknown_speaker["speaker"]:
                # Then we also have a location
                list_of_possible_members = members.loc[(members["term_start"] < unknown_speaker["date"]) &
                            (members["term_end"] > unknown_speaker["date"]) &
                            (members["state"] == state_names[unknown_speaker["speaker"].split(" of ")[1]]) &
                            (members["type"] == ("rep" if unknown_speaker["header.chamber"] == "House" else "sen")) &
                            (members["last_name_lower"] == (unknown_speaker["speaker"].split(" of ")[0].split(" ")[-1].lower()))]\
                            .set_index("full_name_")["bioguide_id"]
            else:
                list_of_possible_members = members.loc[(members["term_start"] < unknown_speaker["date"]) &
                            (members["term_end"] > unknown_speaker["date"]) &
                            (members["type"] == ("rep" if unknown_speaker["header.chamber"] == "House" else "sen")) &
                            (members["last_name_lower"] == (unknown_speaker["speaker"].split(" ")[-1].lower()))]\
                            .set_index("full_name_")["bioguide_id"]

            if len(list_of_possible_members) == 1:
                # Reverse match using name to get bioguide id and set that as the id for all missing rows
                speech_df.loc[speech_df["speaker_bioguide"] == "None", "speaker_bioguide"] = list_of_possible_members.iloc[0]
                speech_df["changed"] = True
            elif len(list_of_possible_members) > 1:
                best_match = process.extractOne(title_row, list_of_possible_members.index.tolist(), scorer=fuzz.token_set_ratio)
                if best_match[1] > 85:
                    # Good match so let's use it
                    # Reverse match using name to get bioguide id and set that as the id for all missing rows
                    speech_df.loc[speech_df["speaker_bioguide"] == "None", "speaker_bioguide"] = list_of_possible_members.to_dict()[best_match[0]]
                    speech_df["changed"] = True

    if len(speech_df) > 0:
        # Group speeches together if by the same person
        speech_df = speech_df.groupby(["speaker", "speaker_bioguide", "id", "doc_title", "date"])\
            .apply(lambda x: " ".join(x.text)).reset_index().rename(columns={0:"body"})

        return speech_df

In [5]:
%%time
from multiprocessing import Pool

files = glob.glob("/media/Stuff/congressional-record/output/**/json/*", recursive=True)

with Pool(8) as pool:
    speeches = pd.concat(list(pool.map(speech_to_df, files)), ignore_index=True).drop("index", axis=1)

CPU times: user 6min 18s, sys: 20.5 s, total: 6min 38s
Wall time: 45min 39s


In [6]:
speeches.join(members[["bioguide_id", "first_name", "last_name", "gender"]]\
                                    .drop_duplicates().set_index("bioguide_id"), on="speaker_bioguide")

,body,date,doc_title,id,speaker,speaker_bioguide,first_name,last_name,gender
0,"Mr. LANGEVIN. Mr. Speaker, I rise in support...",2001-03-30,MARRIAGE PENALTY AND FAMILY TAX RELIEF ACT OF ...,CREC-2001-03-30-pt1-PgE503,Mr. LANGEVIN,L000559,James,Langevin,M
1,"Mr. BOEHLERT. Mr. Speaker, I rise in strong ...",2001-03-30,MARRIAGE PENALTY AND FAMILY TAX RELIEF ACT OF ...,CREC-2001-03-30-pt1-PgE503-2,Mr. BOEHLERT,B000586,Sherwood,Boehlert,M
2,"Mr. LARSON of Connecticut. Mr. Speaker, I ri...",2001-03-30,RECOGNIZING EVAN DOBELLE'S CONTRIBUTIONS TO TH...,CREC-2001-03-30-pt1-PgE504,Mr. LARSON of Connecticut,L000557,John,Larson,M
3,"Mr. SCOTT. Mr. Speaker, in celebration of Wo...",2001-03-30,MAGGIE LENA WALKER,CREC-2001-03-30-pt1-PgE505,Mr. SCOTT,S000185,Robert,Scott,M
4,"Mr. LANGEVIN. Mr. Chairman, I rise today in ...",2001-03-30,"CONCURRENT RESOLUTION ON THE BUDGET, FISCAL YE...",CREC-2001-03-30-pt1-PgE504-2,Mr. LANGEVIN,L000559,James,Langevin,M
5,"Mr. MORAN of Kansas. Mr. Speaker, March Madn...",2001-03-30,CONGRATULATING CLOUD COUNTY COMMUNITY COLLEGE,CREC-2001-03-30-pt1-PgE503-4,Mr. MORAN of Kansas,M000934,Jerry,Moran,M
6,"Mr. CHAMBLISS. Mr. Speaker, I want to recogn...",2001-03-30,"IN RECOGNITION OF CAROLYN CRAYTON, THE FOUNDER...",CREC-2001-03-30-pt1-PgE503-3,Mr. CHAMBLISS,C000286,Saxby,Chambliss,M
7,"Mr. DeFAZIO. Mr. Speaker, this weekend is no...",2001-03-30,STOP THE TIDE OF SUBSIDIZED CANADIAN LUMBER FR...,CREC-2001-03-30-pt1-PgH1350,Mr. DeFAZIO,D000191,Peter,DeFazio,M
8,"Mr. HASTINGS of Florida. Mr. Speaker, I rise...",2001-03-30,ACHIEVEMENTS OF CESAR CHAVEZ,CREC-2001-03-30-pt1-PgE505-2,Mr. HASTINGS of Florida,H000324,Alcee,Hastings,M
9,"Mr. RILEY. Mr. Speaker, I would like to welc...",2001-03-30,DR. ROGER D. WILLMORE,CREC-2001-03-30-pt1-PgH1349-6,Mr. RILEY,R000258,Bob,Riley,M


Separate out speeches into body and metadata dataframes so that we can lazy load the speech bodies later.

In [2]:
import bcolz

# Save data to hdf5
speeches.drop("body", axis=1).to_hdf("speeches_metadata.h5", "metadata", mode="w", format="table")

# Save speeches to bcolz array
bcolz.carray(speeches["body"].str.replace("\n", " "), rootdir="speeches.bcolz", chunklen=10000000, cparams=bcolz.cparams(cname="lz4hc"))

ERROR:root:Line magic function `%file` not found (But cell magic `%%file` exists, did you mean that instead?).


In [8]:
# Save files that have been processed so that in the future we can append to speeches dataframe without duplicating work
pd.Series(files).to_csv("processed_files.csv", index=False)

In [9]:
del speeches